In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/movie-ratings/movie_ratings_df.csv


In [2]:
!pip install findspark
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.0/199.0 KB 15.0 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=2046176c025979c24abb9b36a02d1f2f975d8163f87f611af29213f6236f2f15
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.5
    Uninstalling py4j-0.10.9.5:
      Successfully uninstalled py4j-0.10.9.5


In [3]:
#import spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('recom_sys').getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/25 13:18:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
#import libs
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import *
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import IndexToString
from pyspark.ml.recommendation import ALS

In [5]:
#read the dataset
df = spark.read.csv('/kaggle/input/movie-ratings/movie_ratings_df.csv', header=True, inferSchema=True)
df.show(1)


+------+------------+------+
|userId|       title|rating|
+------+------------+------+
|   196|Kolya (1996)|     3|
+------+------------+------+
only showing top 1 row



In [6]:
#shape of data
print(((df.count()), len(df.columns)))

(100000, 3)


In [7]:
df.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- rating: integer (nullable = true)



In [8]:
df.orderBy(rand()).show(10,False)

+------+-------------------------------+------+
|userId|title                          |rating|
+------+-------------------------------+------+
|537   |Mary Poppins (1964)            |2     |
|566   |Blade Runner (1982)            |4     |
|330   |Hercules (1997)                |4     |
|918   |Paris Is Burning (1990)        |4     |
|148   |Empire Strikes Back, The (1980)|5     |
|178   |North by Northwest (1959)      |3     |
|474   |Christmas Carol, A (1938)      |4     |
|380   |Night on Earth (1991)          |3     |
|200   |Forrest Gump (1994)            |5     |
|753   |Kolya (1996)                   |4     |
+------+-------------------------------+------+
only showing top 10 rows



In [9]:
#checking the ratings per user
df.groupby('userid').count().orderBy('count', ascending=False).show(10,False)

+------+-----+
|userid|count|
+------+-----+
|405   |737  |
|655   |685  |
|13    |636  |
|450   |540  |
|276   |518  |
|416   |493  |
|537   |490  |
|303   |484  |
|234   |480  |
|393   |448  |
+------+-----+
only showing top 10 rows



In [10]:
#checking the ratings per user
df.groupby('userid').count().orderBy('count', ascending=True).show(10,False)

+------+-----+
|userid|count|
+------+-----+
|19    |20   |
|143   |20   |
|309   |20   |
|34    |20   |
|202   |20   |
|732   |20   |
|441   |20   |
|685   |20   |
|824   |20   |
|631   |20   |
+------+-----+
only showing top 10 rows



In [11]:
#movies that is most rated
df.groupby('Title').count().orderBy('count', ascending=False).show(10, False)

+-----------------------------+-----+
|Title                        |count|
+-----------------------------+-----+
|Star Wars (1977)             |583  |
|Contact (1997)               |509  |
|Fargo (1996)                 |508  |
|Return of the Jedi (1983)    |507  |
|Liar Liar (1997)             |485  |
|English Patient, The (1996)  |481  |
|Scream (1996)                |478  |
|Toy Story (1995)             |452  |
|Air Force One (1997)         |431  |
|Independence Day (ID4) (1996)|429  |
+-----------------------------+-----+
only showing top 10 rows



In [12]:
#least rated movie
df.groupby('Title').count().orderBy('count', ascending=True).show(10, False)

+------------------------------------------+-----+
|Title                                     |count|
+------------------------------------------+-----+
|Modern Affair, A (1995)                   |1    |
|Tigrero: A Film That Was Never Made (1994)|1    |
|JLG/JLG - autoportrait de d�cembre (1994) |1    |
|Target (1995)                             |1    |
|Vie est belle, La (Life is Rosey) (1987)  |1    |
|Fear, The (1995)                          |1    |
|Next Step, The (1995)                     |1    |
|Mad Dog Time (1996)                       |1    |
|Leopard Son, The (1996)                   |1    |
|Lashou shentan (1992)                     |1    |
+------------------------------------------+-----+
only showing top 10 rows



In [13]:
stringIndexer = StringIndexer(inputCol='title', outputCol='title_new').fit(df)
#model = stringIndexer.fit(df)
indexed = stringIndexer.transform(df)

In [14]:
indexed.show(10)

+------+------------+------+---------+
|userId|       title|rating|title_new|
+------+------------+------+---------+
|   196|Kolya (1996)|     3|    287.0|
|    63|Kolya (1996)|     3|    287.0|
|   226|Kolya (1996)|     5|    287.0|
|   154|Kolya (1996)|     3|    287.0|
|   306|Kolya (1996)|     5|    287.0|
|   296|Kolya (1996)|     4|    287.0|
|    34|Kolya (1996)|     5|    287.0|
|   271|Kolya (1996)|     4|    287.0|
|   201|Kolya (1996)|     4|    287.0|
|   209|Kolya (1996)|     4|    287.0|
+------+------------+------+---------+
only showing top 10 rows



In [15]:
indexed.groupby('title_new').count().orderBy('count',ascending=False).show(10,False)

+---------+-----+
|title_new|count|
+---------+-----+
|0.0      |583  |
|1.0      |509  |
|2.0      |508  |
|3.0      |507  |
|4.0      |485  |
|5.0      |481  |
|6.0      |478  |
|7.0      |452  |
|8.0      |431  |
|9.0      |429  |
+---------+-----+
only showing top 10 rows



In [16]:
traindf,testdf = indexed.randomSplit([0.75,0.25], seed=1)


In [17]:
print(((traindf.count()), len(traindf.columns)))

(74839, 4)


In [18]:
print(((testdf.count()), len(testdf.columns)))

(25161, 4)


In [19]:
rec= ALS(maxIter=10,regParam=0.01, userCol='userId', itemCol='title_new',ratingCol='rating', nonnegative=True,coldStartStrategy="drop")
rec_model = rec.fit(traindf)